In [13]:
# Initialize an empty dictionary to store the results
result_dict = {}

# Open the FMT file with the given name for reading
f = open("BATERR.FMT", "r")

# Initialize a counter to track LITERAL occurrences
batch_count = 0
store_count = 0
literal_count = 0
crlf_count = 0

# Loop through each line in the file
for line in f:
  # Check if the first character of the line is a letter (indicating a field)
  if line[0].isalpha():
    
    # Remove any leading and trailing whitespaces from the line
    line = line.strip()
    
    # If the line matches the specific string "RECORD_TYPE BATCH_LINE", skip to the next iteration
    if line == "RECORD_TYPE BATCH_LINE":
        continue
    
    # Find the position of the '#' character (indicating the start of a comment)
    pound_loc = line.find("#")
    
    # If a comment is found, remove it from the line
    if pound_loc != -1:
        line = line[:pound_loc]
    
    # Split the line to extract the key (field name)
    key = line.split()[0]
    
    # Extract the value (field format) by removing the key from the line and stripping whitespaces
    value = line.replace(key, "").strip()
    
    # If the key is "BATCH.BATCH_NUMBER", append the current literal_count to the key and increment the count
    if key == "BATCH.BATCH_NUMBER":
        key = f"{key}{batch_count}"
        batch_count += 1

    # If the key is "STORE.STORE_NUMBER", append the current literal_count to the key and increment the count
    if key == "STORE.STORE_NUMBER":
        key = f"{key}{store_count}"
        store_count += 1  

    # If the key is "LITERAL", append the current literal_count to the key and increment the count
    if key == "LITERAL":
        key = f"{key}{literal_count}"
        literal_count += 1

    # If the key is "CRLF", append the current literal_count to the key and increment the count
    if key == "CRLF":
        key = f"{key}{crlf_count}"
        crlf_count += 1
    
    # Add the key-value pair to the result dictionary
    result_dict[key] = value

    
result_dict

{'BATCH.BATCH_NUMBER0': '9',
 'LITERAL0': '"      "',
 'LITERAL1': '"HDR"',
 'LITERAL2': '"    "',
 'STORE.DIVISION_NUMBER': '3',
 'LITERAL3': '"00"',
 'STORE.STORE_NUMBER0': '4',
 'LITERAL4': '"                "',
 'LITERAL5': '"    "',
 'CRLF0': '2',
 'BATCH.BATCH_NUMBER1': '9',
 'BATCH.HOST_SEQ': '9',
 'LITERAL6': '"0"',
 'BATCH_LINE.TABLE': '2',
 'BATCH_OP': '1',
 'LITERAL7': '"0"',
 'BATCH_LINE.ERR_CODE': '5',
 'LITERAL8': '"00"',
 'STORE.STORE_NUMBER1': '4',
 'PRIMARY_KEY': '21',
 'CRLF1': '2'}

In [14]:
#further processing for length of fields and transforming str nums to ints
for key, value in result_dict.items():
    if value.startswith('"') and value.endswith('"'):
        str_value = value.strip('"')
        if str_value.isdigit():
            result_dict[key] = len(str_value)
        else:
            result_dict[key] = len(str_value)
    elif value.isdigit():
        result_dict[key] = int(value)

In [16]:
hdr_keys = [
    "BATCH.BATCH_NUMBER0",
    "LITERAL0",
    "LITERAL1",
    "LITERAL2",
    "STORE.DIVISION_NUMBER",
    "LITERAL3",
    "STORE.STORE_NUMBER0",
    "LITERAL4",
    "LITERAL5"
]
hdr_dict  = result_dict
hdr_dict = {key: result_dict[key] for key in hdr_keys}
hdr_dict["BATCH.BATCH_NUMBER0"] = 6 
hdr_dict

{'BATCH.BATCH_NUMBER0': 6,
 'LITERAL0': 6,
 'LITERAL1': 3,
 'LITERAL2': 4,
 'STORE.DIVISION_NUMBER': 3,
 'LITERAL3': 2,
 'STORE.STORE_NUMBER0': 4,
 'LITERAL4': 16,
 'LITERAL5': 4}

In [35]:
import pandas as pd

hdr_df = pd.DataFrame()

for line in f:
    if "HDR" in line:
        result_dict = {}
        for key, field_len in hdr_dict.items():
            value = line[:field_len]
            line=line[field_len:]
            if value.isspace():
                value = f'"{value}"'
            result_dict[key]=[value]
        hdr_df = pd.concat([
            pd.DataFrame(result_dict), hdr_df
        ])
        hdr_df = hdr_df.reset_index()

hdr_df

""


In [41]:
table_keys = [
    # "BATCH.BATCH_NUMBER1",
    "BATCH.HOST_SEQ",
    "LITERAL6",
    "BATCH_LINE.TABLE",
    "BATCH_OP",
    "LITERAL7",
    "BATCH_LINE.ERR_CODE",
    "LITERAL8",
    "STORE.STORE_NUMBER1",
    "PRIMARY_KEY",
    "CRLF1"
]
table_dict  = result_dict
table_dict = {key: result_dict[key] for key in table_keys}
table_dict["BATCH.BATCH_NUMBER1"] = 6 
table_dict

KeyError: 'BATCH.HOST_SEQ'

In [36]:
table_df = pd.DataFrame()
f = open("BATERRCT.DAT", "r")

# if "SUP" in line:
for line in f:
    if "HDR" not in line:
        result_dict = {}
        for key, field_len in table_dict.items():
            value = line[:field_len]
            line=line[field_len:]
            if value.isspace():
                value = f'"{value}"'
            result_dict[key]=[value]
        table_df = pd.concat([
            pd.DataFrame(result_dict), table_df
        ])
        table_df = table_df.reset_index()

table_df

TypeError: slice indices must be integers or None or have an __index__ method